In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the Google Sheets published page
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRmygydebEruBWt5L_vYwpexFqNXV511pkRKpPu5GWlezhK5LJPutCDOMxES3b4r52E8n9o5WbuDbVE/pubhtml"

# Get the webpage content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all table rows
rows = soup.find_all('tr')

# Initialize lists to store data
dates = []
names = []

# Extract data from rows
for row in rows[1:]:  # Skip header row
    cells = row.find_all('td')
    if len(cells) >= 2:  # Ensure row has enough cells
        date = cells[0].text.strip()
        name = cells[1].text.strip()
        
        if date and name:  # Only add if both date and name exist
            dates.append(date)
            names.append(name)

# Create DataFrame
df = pd.DataFrame({
    'Date': dates,
    'Name': names
})

# Display the first few rows
print(df.head(10))

        Date                                            Name
0  INSTAGRAM                                       @DONAU115
1      EMAIL                                INFO@DONAU115.DE
2   FACEBOOK                                 FB.COM/DONAU115
3      SET 1          DAVE J., DENISE, LAURENT, COCO, MATICO
4      SET 2           SOPHIE, SASCHA, AUGUSTO, DAVIDE, VERA
5      SET 3  KARLOTTA, LINNEA, VERONICA, CHRISTIAN, JOE, ED
6  DI.22.OKT                                TWO-SONG TUESDAY
7  MI.23.OKT              ELI WALTZ & DOMINO LEWIS⎢NEIGHBOUR
8  DO.24.OKT              THE KILLING POPES (GB / GER / JAP)
9  FR.25.OKT                                            EYRA


In [17]:
# Convert German dates to datetime objects
def convert_german_date(date_str):
    try:
        # Split the date string by dots
        parts = date_str.split('.')
        if len(parts) != 3:
            return pd.NA
            
        weekday, day, month = parts
        
        # Dictionary for German and English month abbreviations
        months = {
            # German
            'JAN': '01', 'FEB': '02', 'MÄR': '03', 'APR': '04',
            'MAI': '05', 'JUN': '06', 'JUL': '07', 'AUG': '08',
            'SEP': '09', 'OKT': '10', 'NOV': '11', 'DEZ': '12',
            # English
            'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04',
            'MAY': '05', 'JUN': '06', 'JUL': '07', 'AUG': '08',
            'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'
        }
        
        # Convert month to number
        month = months[month.upper()]
        
        # Create date string in format DD-MM-YYYY HH:MM (assuming current year)
        current_year = pd.Timestamp.now().year
        date_string = f"{day}-{month}-{current_year} 20:30"
        
        # Convert to datetime including time
        return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M')
    except (ValueError, KeyError, IndexError):
        return pd.NA

# Apply the conversion to the Date column
df['Date'] = df['Date'].apply(convert_german_date)



In [18]:
# Filter out rows where Date is NaT (Not a Time) - these are invalid dates
df = df.dropna(subset=['Date'])

# Add constant link column
df['Link'] = "http://www.donau115.de/"


# Display the first few rows after date conversion
print(df.head())

print(df.info())



                   Date                                Name  \
6   2024-10-22 20:30:00                    TWO-SONG TUESDAY   
7   2024-10-23 20:30:00  ELI WALTZ & DOMINO LEWIS⎢NEIGHBOUR   
8   2024-10-24 20:30:00  THE KILLING POPES (GB / GER / JAP)   
9   2024-10-25 20:30:00                                EYRA   
10  2024-10-26 20:30:00                 INGI BJARNI QUINTET   

                       Link  
6   http://www.donau115.de/  
7   http://www.donau115.de/  
8   http://www.donau115.de/  
9   http://www.donau115.de/  
10  http://www.donau115.de/  
<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 6 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    38 non-null     object
 1   Name    38 non-null     object
 2   Link    38 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB
None


In [19]:
# add venue name and geolocation
df['Venue'] = "Donau115"
df['Location'] = "Donau115, Donaustraße 115, 12043 Berlin"


In [20]:
df.to_csv('events.csv', index=False)




# Create Calendar

In [22]:
import pandas as pd
from datetime import datetime, timedelta
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

# OAuth 2.0 flow
SCOPES = ['https://www.googleapis.com/auth/calendar']
flow = InstalledAppFlow.from_client_secrets_file(
    'credentials_google_calendar_api.json',
    SCOPES
)
creds = flow.run_local_server(port=0)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=865234066123-d6f0ga34jrh1pkmoucfph44ugfc84d42.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57664%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=K7v6mf7AZmv4ULG2J1kg9pcRT3wFoq&access_type=offline


In [21]:
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build

# SCOPES = ['https://www.googleapis.com/auth/calendar']

# def create_events_calendar():
   
#     # Build the service
#     service = build('calendar', 'v3', credentials=creds)
    
#     # Create new calendar
#     calendar = {
#         'summary': 'events',
#         'timeZone': 'Europe/Berlin'
#     }
    
#     try:
#         created_calendar = service.calendars().insert(body=calendar).execute()
#         print(f'Created calendar: {created_calendar["id"]}')
#         return created_calendar['id']
#     except Exception as e:
#         print(f'An error occurred: {e}')
#         return None

# # Create the calendar
# calendar_id = create_events_calendar()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=865234066123-d6f0ga34jrh1pkmoucfph44ugfc84d42.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57651%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=zryhuYxcARsxbLAd1WQqG4atJje8I0&access_type=offline
Created calendar: 2804584b6be17838c086911b4d4325f3da88786e85a019b84a52163c2ea45e79@group.calendar.google.com


In [23]:
# Load the CSV file
df = pd.read_csv('events.csv')

# Build the service using the credentials from previous cell
service = build('calendar', 'v3', credentials=creds)

# Find the 'events' calendar
calendar_list = service.calendarList().list().execute()
events_calendar = next((calendar for calendar in calendar_list['items'] if calendar['summary'] == 'events'), None)

if not events_calendar:
    print("Calendar 'events' not found. Please make sure it exists.")
else:
    calendar_id = events_calendar['id']

    # Get list of existing events and their IDs
    existing_events = {}
    page_token = None
    while True:
        events = service.events().list(calendarId=calendar_id, pageToken=page_token).execute()
        for event in events['items']:
            existing_events[event['summary']] = event['id']
        page_token = events.get('nextPageToken')
        if not page_token:
            break

    # Add or update each event from the CSV
    for _, row in df.iterrows():
        # Parse the date
        event_date = datetime.strptime(row['Date'], '%Y-%m-%d %H:%M:%S')
        
        # Set event duration to 2 hours
        event_end = event_date + timedelta(hours=2)
        
        # Create event body
        event = {
            'summary': row['Name'],
            'description': f"More info: {row['Link']}" if row['Link'] else "",
            'location': f"{row['Location']})",  # Show venue name with location link
            'start': {
                'dateTime': event_date.strftime('%Y-%m-%dT%H:%M:%S'),
                'timeZone': 'Europe/Berlin',
            },
            'end': {
                'dateTime': event_end.strftime('%Y-%m-%dT%H:%M:%S'),
                'timeZone': 'Europe/Berlin',
            },
        }
        
        try:
            if row['Name'] in existing_events:
                # Update existing event
                service.events().update(
                    calendarId=calendar_id,
                    eventId=existing_events[row['Name']],
                    body=event
                ).execute()
                print(f"Updated event: {row['Name']}")
            else:
                # Create new event
                service.events().insert(calendarId=calendar_id, body=event).execute()
                print(f"Added event: {row['Name']}")
        except Exception as e:
            print(f"Error processing event {row['Name']}: {e}")

Updated event: TWO-SONG TUESDAY
Updated event: ELI WALTZ & DOMINO LEWIS⎢NEIGHBOUR
Updated event: THE KILLING POPES (GB / GER / JAP)
Updated event: EYRA
Updated event: INGI BJARNI QUINTET
Updated event: SUSI SHOCK
Updated event: TWO-SONG TUESDAY
Updated event: KENNY WARREN TRIO
Updated event: AMINO THRIFT
Updated event: BRUNO BODE QUARTET
Updated event: XLUSH
Updated event: RANDY INGRAM TRIO
Updated event: TWO-SONG TUESDAY
Updated event: JIB WEDNESDAY W/ IRMA NEUMÜLLER & SETH SJÖSTRÖM
Updated event: MICHAËL ATTIAS KARDAMON SPRING
Updated event: BYE PARULA
Updated event: DREAM BIG FISH
Updated event: TWO-SONG TUESDAY
Updated event: KLEPPAN
Updated event: VOLKER MEITZ - NINE DUOS (ALBUM RELEASE)
Updated event: KAISER POMMES (NISSEN/TRANBERG/STEIDLE/STEFANSSON)
Updated event: ASTRUP/DONKIN/WANKEL
Updated event: TWO-SONG TUESDAY
Updated event: JULIA PERMINOVA TRIO
Updated event: FELIX AMBACH QUARTET
Updated event: WANJA SLAVIN
Updated event: DEVIN GRAY
Updated event: TWO-SONG TUESDAY
Update